In [1]:
!pip install numba
!pip install matplotlib
!pip install ray
!pip install tqdm
!pip install more_itertools
!pip install sklearn
!pip install pydot

     |████████████████████████████████| 3.1 MB 5.5 MB/s eta 0:00:01
     |████████████████████████████████| 24.6 MB 22.1 MB/s eta 0:00:01    |█████▌                          | 4.2 MB 22.1 MB/s eta 0:00:01     |██████████████████████████████▎ | 23.3 MB 22.1 MB/s eta 0:00:01
     |████████████████████████████████| 11.6 MB 5.6 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 45.1 MB/s eta 0:00:01
     |████████████████████████████████| 2.2 MB 42.8 MB/s eta 0:00:01
     |████████████████████████████████| 23.1 MB 5.3 MB/s eta 0:00:01
     |████████████████████████████████| 45 kB 6.4 MB/s  eta 0:00:01
     |████████████████████████████████| 1.3 MB 34.1 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 10.7 MB/s eta 0:00:01
     |████████████████████████████████| 269 kB 29.9 MB/s eta 0:00:01
     |████████████████████████████████| 127 kB 42.0 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 7.1 MB/s  eta 0:00:01
     |████████████████████████████████|

In [84]:
import numpy as np
import ray
import more_itertools
from tqdm.notebook import tqdm 
import numpy as np
from numba import jit ,f8
import os
import h5py

from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import *
from tensorflow.keras.layers import *
import tensorflow


from datetime import datetime
import time
import sklearn
import sklearn.metrics

## Solver & Data generator

In [83]:
@jit(nopython=True)
def generate_grid(n,bc,ic=0):
    #n=> Number of interior nodes

    A = np.ones((n+2,n+2),dtype=np.float32) * ic
    A[0,0]=A[-1,-1]=A[0,-1]=A[-1,0]=0
    A[0,1:-1]=bc[2]    # switch the top and bottom wall since we start the iterations from top
    A[1:-1,-1]=bc[1]
    A[-1,1:-1]=bc[0]
    A[1:-1,0]=bc[3]

    return A

@jit(nopython=True)
def update_grid_col(grid,x,n):
    #first half step update
    #x col values ;
    #n col number 
    grid[1:-1,n] = x
    return grid

@jit(nopython=True)
def update_grid_row(grid,x,n):
    #second half step update
    #x row values ;
    #n col number 
    grid[n,1:-1] = x
    return grid

@jit(nopython=True)
def calculate_first_half(A,i,j,lam):
    #calculate the ADI explicit part of the equation first half step
    if j==1 :
        return  lam * A[j][i-1] + 2 *(1-lam) * A[j][i] + lam * A[j][i+1] + lam * A[j-1][i]
    elif j== A.shape[0]-2 :
        return  lam * A[j][i-1] + 2 *(1-lam) * A[j][i] + lam * A[j][i+1] + lam * A[j+1][i]
    else  :
        return  lam * A[j][i-1] + 2 *(1-lam) * A[j][i] + lam * A[j][i+1]

@jit(nopython=True)
def calculate_second_half(A,i,j,lam):
    #calculate the ADI explicit part of the equation second half step
    if i==1 :
        return  lam * A[j-1][i] + 2 *(1-lam) * A[j][i] + lam * A[j+1][i] + lam * A[j][i-1]
    elif i== A.shape[0]-2 :
        return  lam * A[j-1][i] + 2 *(1-lam) * A[j][i] + lam * A[j+1][i] + lam * A[j][i+1]
    else  :
        return  lam * A[j-1][i] + 2 *(1-lam) * A[j][i] + lam * A[j+1][i]

@jit(nopython=True)
def generate_TDM(Lambda,N=3):
    a = np.ones(N-1,dtype=np.float32)*-Lambda 
    b =np.ones(N,dtype=np.float32) * 2*(Lambda+1) 
    return a,b,a

@jit(f8[:](f8[:],f8[:],f8[:],f8[:]))
def TDMA_solver(a0,b0,c0,d0):
    a =np.copy(a0)
    b =np.copy(b0)
    c =np.copy(c0)
    d =np.copy(d0)
    ld = len(d)

    for i in range(1,ld):
        w    = a[i-1]/b[i-1]
        b[i] = b[i]- w * c[i-1]
        d[i] = d[i] -w * d[i-1]
    
    R=b
    R[-1]=d[-1]/b[-1]
    
    for i in range(ld-2,-1,-1):
        R[i]= (d[i]-c[i]*R[i+1]) /b[i]
        
    return R

@jit(nopython=True)
def ADI_first_half_step(grid,Lambda,a,b,c):
    #apply ADI for single step

    N = grid.shape[0]

    ##First half step
    for i in range(1,N-1):

        # initialize explicit side of equation to zeros
        d = np.zeros((N-2))

        #move vertically implcitly and calculate horizontally explicitly
        for j in range(1,N-1):
            d[j-1] =calculate_first_half(grid,i,j,Lambda)

        x = TDMA_solver(a,b,c,d)

        grid = update_grid_col(grid,x,i)

    return grid

@jit(nopython=True)
def ADI_second_half_step(grid,Lambda,a,b,c):
    #apply ADI for single step

    N = grid.shape[1]

    ##Second half step
    for i in range(1,N-1):

        # initialize explicit side of equation to zeros
        d = np.zeros((N-2))

        #move horizontally implcitly and calculate vertically explicitly
        for j in range(1,N-1):
            d[j-1] =calculate_second_half(grid,j,i,Lambda)

        x = TDMA_solver(a,b,c,d)

        grid = update_grid_row(grid,x,i)

    return grid

@jit(nopython=True)
def solve(grid,Lambda=1,iters=1,steps=False) :
    if steps :
        #save intermeidate steps

        #(frames,height,width)
        grids = np.zeros((iters+1, grid.shape[0],grid.shape[1]),dtype=np.float32)
        grids[0,:,:]=grid
        a,b,c = generate_TDM(Lambda,grid.shape[0]-2)
        #apply ADI iteratively
        for i in range(1,iters+1):
            grids[i,:,:] = ADI_first_half_step(grid,Lambda,a,b,c)
            grids[i,:,:] = ADI_second_half_step(grid,Lambda,a,b,c)


    else : 
        grids = np.zeros((2,grid.shape[0],grid.shape[1]),dtype=np.float32)
        #Show final step only
        grids[0,:,:]=grid
        a,b,c = generate_TDM(Lambda,grid.shape[0]-2)
        #apply ADI iteratively
        for i in range(iters):
            grids[1,:,:] = ADI_first_half_step(grid,Lambda,a,b,c)
            grids[1,:,:] = ADI_second_half_step(grid,Lambda,a,b,c)

    return grids




def pad_grids(grids,Lambda):
    grids[:,0,0]=grids[:,-1,0]=grids[:,0,-1]=grids[:,-1,-1]=Lambda * 1000
    return grids

@ray.remote
def solve_permutation(n,iters,permutation):
    Lambda,bc1,bc2,bc3,bc4,ic = permutation
    grid = generate_grid(n-2,bc=(bc1,bc2,bc3,bc4),ic=ic)
    ADIsoltuion = solve(grid.copy(),Lambda = Lambda,iters =iters ,steps=True)
    return  np.array(pad_grids(ADIsoltuion,Lambda)).reshape(ADIsoltuion.shape[0],ADIsoltuion.shape[1],ADIsoltuion.shape[2],1)


def generate_data(N,iters,permutations):
    '''
    Input : 
    N            : size of grid
    iters        : max iterations done by solver
    permutations : the solution parameters as a set of permutation (Lambda,bc1,..bc4,ic0)
    
    Output:
    solution with shape (iters+1,N,N)
    '''
    data = [(solve_permutation.remote(N,iters,i)) for i in (permutations) ]
    return np.array([ray.get(datalet) for datalet in (data)])
    

def generate_data_random_permutations(lR=(0,0.25),tR=(0,1000),batches =1,batch_size=32,seed =42,split=1 ):
    '''
    Input:
    *Lambda range
    *Temperature range
    *Size of data
    
    Output:
    *Generate a generator of size with elements of (Lambda,bc1,..bc4,ic0)
    '''
    np.random.seed(seed);
    lr = np.random.randint(low = tR[0] , high = tR[1] ,size=(batches,batch_size,6)).astype('float')
    lr[:,:,0] = ((lR[1]-lR[0])*(lr[:,:,0]-tR[0]))/(tR[1]-tR[0])
    return lr


def generate_data_batches(N=50,
                          lR=(0,0.5),
                          tR=(0,100),
                          max_iters=10,
                          seed=42,
                          steps=1,
                          step_size=1,
                          batch_size=32,
                          batches=100,
                          progress=True,
                          key_bias =0,
                          save_file = None):
    '''
    return dictionary with key of the batch number
    '''
    if save_file is not None : 
        hf = h5py.File(save_file,'w')

    np.random.seed(seed)
    perms = generate_data_random_permutations(lR=lR,tR=tR,batch_size=batch_size,batches=batches,seed=seed)
    iters_list=  np.random.randint(low=step_size,high=max_iters-step_size*(steps-1)+1,size=batches)

    #scaling 
    mean = (tR[1]-tR[0])/2
    std  = mean/2
    data={}
    
    for batch_num in tqdm(range(batches)):
        
        iter_n = iters_list[batch_num]
        
        generated_data = generate_data(N,iter_n+steps*step_size,perms[batch_num])
        extract_index = np.arange(iter_n-step_size,iter_n+step_size*steps,step_size)
#         yield(extract_index)
        generated_data = generated_data[:,extract_index,:,:,:]
        
        if save_file is None :
            data[f'{batch_num + key_bias}'] = (generated_data -mean) /std
        else : 
            hf.create_dataset(f'{batch_num}',data = data[batch_num] , compression ='gzip')
    
    if save_file is not None :hf.close()
    else : return data

    
def data_generator(data):
    '''
    input : data dictionary (batch number :5D tensor data)
    output: input , target values
    '''
    batches = len(data.keys())
    batch_size = len(data['0'])
    batch_counter= 0
    
    while True:
        x,y = data[f'{batch_counter}'][:,:-1,:,:,:],data[f'{batch_counter}'][:,-1:,:,:,:]

        batch_counter += 1
        yield x,y
        if batch_counter == batches:batch_counter = 0


In [2]:
ray.init()

2020-11-14 05:51:19,448	INFO services.py:1092 -- View the Ray dashboard at http://127.0.0.1:8265
2020-11-14 05:51:19,453	WARNING services.py:1560 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 16106127360 bytes available. This may slow down performance! You may be able to free up space by deleting files in /dev/shm or terminating any running plasma_store_server processes. If you are inside a Docker container, you may need to pass an argument with the flag '--shm-size' to 'docker run'.


{'node_ip_address': '172.17.0.6',
 'raylet_ip_address': '172.17.0.6',
 'redis_address': '172.17.0.6:6379',
 'object_store_address': '/tmp/ray/session_2020-11-14_05-51-18_608336_6806/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-11-14_05-51-18_608336_6806/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2020-11-14_05-51-18_608336_6806',
 'metrics_export_port': 43426,
 'node_id': '96a989bcf3902062d87392efb24bcd142d40429a'}

In [145]:
train_data_batches =generate_data_batches(N=12,
                                          lR=(0,1),
                                          tR=(0,1_000),
                                          max_iters=1000,
                                          seed=42,
                                          batch_size=32,
                                          step_size= 100,
                                          steps=1,
                                          batches=9000,
                                          progress=True)
#                                           save_file = 'N=100x100 batch_size=64 batches=5000 max_iters=1000')

In [146]:
train_data_batches_bias =generate_data_batches(N=12,
                                          lR=(0,1),
                                          tR=(0,1_000),
                                          max_iters=120,
                                          seed=42,
                                          batch_size=32,
                                          step_size= 100,
                                          steps=1,
                                          batches =1_000,
                                          key_bias = 9_000,
                                          progress = True)
#                                           save_file = 'N=100x100 batch_size=64 batches=5000 max_iters=1000')

In [147]:
validation_data_batches =generate_data_batches(N=12,
                                              lR=(0,1),
                                              tR=(0,1_000),
                                              max_iters=1000,
                                              seed=0,
                                              batch_size=32,
                                              step_size= 100,
                                              steps=1,
                                              batches=50,
                                              progress=True)

## Model

In [155]:
def conv(x,f,k):
    x = TimeDistributed(Conv2D(f,(k,k),strides=1,padding='same',kernel_initializer='glorot_uniform',activation=LeakyReLU()))(x)
    return x
    
def deconv(x,f,k):
    x = TimeDistributed(Conv2DTranspose(f,(k,k),strides=1,padding='same',kernel_initializer='glorot_uniform',activation=LeakyReLU()))(x)
    return x
    
    
def dense_block(tensor, f, r,k):
    for _ in range(r):
        x = conv(tensor, f=4*f, k=1)
        x = conv(x, f=f, k=k)
        tensor = Concatenate()([tensor, x])
    return tensor

def inv_dense_block(tensor, f, r,k):
    for _ in range(r):
        x = deconv(tensor, f=4*f, k=1)
        x = deconv(x, f=f, k=k)
        tensor = Concatenate()([tensor, x])
    return tensor

def transition(x,s):
    ff = int(tensorflow.keras.backend.int_shape(x)[-1] * 0.5)
    m0 = TimeDistributed(Conv2D(ff,(1,1),strides=2*s,padding='same',kernel_initializer='glorot_uniform',activation=LeakyReLU()))(x)
    return m0

def inv_transition(x,s):
    ff = int(tensorflow.keras.backend.int_shape(x)[-1] * 0.5)
    m0 = TimeDistributed(Conv2DTranspose(ff,(1,1),strides=2*s,padding='same',kernel_initializer='glorot_uniform',activation=LeakyReLU()))(x)
    return m0


def dfn():
    
    k=3
    s=1;
    LR=1e-4
    
    r1,r2,r3 = 2 , 4 ,8
    f0,f1,f2,f3 = 128,32,32,32
    l1,l2 = 128 ,64

    x = Input(shape=(None, None,None, 1))
    c0 = TimeDistributed(Conv2D(f0,(k,k),strides=1,padding='same',kernel_initializer='glorot_uniform',activation=LeakyReLU()))(x)
########################################################################################################    
    e1 = dense_block(c0,f1,r=r1,k=k);m1 = transition(e1,s)
    e2 = dense_block(m1,f2,r=r2,k=k);m2 = transition(e2,s)
    e3 = dense_block(m2,f3,r=r3,k=k);
########################################################################################################
    e = ConvLSTM2D(l1,(2,2),padding='same',return_sequences=True)(e3)
    b = ConvLSTM2D(l2,(2,2),padding='same',return_sequences=True)(e)
    d = ConvLSTM2D(l1,(2,2),padding='same',return_sequences=True)(b)
########################################################################################################
    d1 = inv_dense_block(d ,f3,r=r3,k=k);m1 = inv_transition(d1,s)
    d2 = inv_dense_block(m1,f2,r=r2,k=k);m2 = inv_transition(d2,s)
    d3 = inv_dense_block(m2,f1,r=r1,k=k);
########################################################################################################
    out = conv(d3,f=1,k=1)
    model = Model(x,out)
    optimizer = Adam(learning_rate=LR)
    model.compile(loss='mae',optimizer=optimizer,metrics=['mse'])
    return model

model = dfn()
# model.summary()
# keras.utils.plot_model(model,show_shapes=True)

## Train

In [156]:
history={}
i = 100

In [157]:
csv_logger = tensorflow.keras.callbacks.CSVLogger('train step=100 12x12.log')
early_stopping = tensorflow.keras.callbacks.EarlyStopping(monitor='loss',min_delta=5e-5, patience=5, verbose=1, mode='auto',baseline=None, restore_best_weights=False)
reduce_lr_callback = tensorflow.keras.callbacks.ReduceLROnPlateau(monitor = 'loss',factor = 0.5,patience = 3,verbose = 1,cooldown=1,min_delta = 1e-4,min_lr=1e-8 )
model_checkpoint_callback = tensorflow.keras.callbacks.ModelCheckpoint('step=100 12x12.h5', monitor='loss', verbose=1, save_best_only=False,save_weights_only=False, mode='auto', save_freq='epoch',)
model.fit(data_generator({**train_data_batches,**train_data_batches_bias}),
          validation_data=data_generator(validation_data_batches),
          steps_per_epoch=len({**train_data_batches,**train_data_batches_bias}),
          validation_steps=len(validation_data_batches),
          verbose=1,
          epochs=100,
          callbacks=[reduce_lr_callback,early_stopping,csv_logger,model_checkpoint_callback],
         )

Epoch 1/100
10000/10000 [==============================] - 1196s 117ms/step - loss: 0.1300 - mse: 0.1154 - val_loss: 0.0571 - val_mse: 0.0426

Epoch 00001: saving model to step=100 12x12.h5
Epoch 2/100
10000/10000 [==============================] - 1150s 115ms/step - loss: 0.0469 - mse: 0.0420 - val_loss: 0.0479 - val_mse: 0.0407

Epoch 00002: saving model to step=100 12x12.h5
Epoch 3/100
10000/10000 [==============================] - 1132s 113ms/step - loss: 0.0430 - mse: 0.0389 - val_loss: 0.0277 - val_mse: 0.0160

Epoch 00003: saving model to step=100 12x12.h5
Epoch 4/100
10000/10000 [==============================] - 1164s 116ms/step - loss: 0.0176 - mse: 0.0053 - val_loss: 0.0134 - val_mse: 0.0012

Epoch 00004: saving model to step=100 12x12.h5
Epoch 5/100
10000/10000 [==============================] - 1174s 117ms/step - loss: 0.0109 - mse: 0.0012 - val_loss: 0.0099 - val_mse: 0.0011

Epoch 00005: saving model to step=100 12x12.h5
Epoch 6/100
10000/10000 [=========================

## Save model

In [158]:
model.save('step=100 12x12 loss=9.0113e-4 9.9932e-4.h5')

## Visualize data

In [10]:
import sklearn.metrics
import sklearn

In [139]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid

def show_heat_maps(*grids,annotate=False,save=None , rc=None,figsize=(30,10),cbar_location='left'):
    fig = plt.figure(figsize=figsize)
    if rc is  None : rc=(1,len(grids))
    grid = ImageGrid(fig, 111, nrows_ncols=rc,axes_pad=0.25,share_all=True,cbar_location=cbar_location,cbar_mode="single",cbar_size="5%",cbar_pad=0.25,)
    for ax,g in zip(grid,grids):
        im = ax.imshow(g[0]) ; 
        ax.title.set_text(g[1])
        if annotate:
            N,M = int(g[0].shape[0]),int(g[0].shape[1])
            for k in range(N):
                for j in range(M):
                    text1 = ax.text(j, k, np.round(g[0][k, j],1),ha="center", va="center", color="w",fontsize=20)


        ax.cax.colorbar(im)
        ax.cax.toggle_label(True)
    
    if save is not None : plt.savefig(save)
    plt.show()


In [86]:
import  tensorflow
loaded_model = model #tensorflow.keras.models.load_model('48x48 loss=6.152e-4 7.5453e-4.h5')
loaded_model2 = model # model_scaled #tensorflow.keras.models.load_model('bias 100x100 step=10 loss=0.0012 model 27th october model.h5')

# loaded_model2 = keras.models.load_model('100x100 step=10 loss=0.008 model 17th october model.h5')

In [140]:
import ipywidgets
layout = ipywidgets.Layout(width= '100%',height='20px')

bc1 = ipywidgets.IntSlider(min=0,max=1000,value = 600,step=1,description='bc1 # ' ,layout=layout,continuous_update=False)
bc2 = ipywidgets.IntSlider(min=0,max=1000,value=500,step=1,description='bc2 # ' ,layout=layout,continuous_update=False)
bc3 = ipywidgets.IntSlider(min=0,max=1000,value=100,step=1,description='bc3 # ' ,layout=layout,continuous_update=False)
bc4 = ipywidgets.IntSlider(min=0,max=1000,step=1,value=400,description='bc4 # ' ,layout=layout,continuous_update=False)
ic0 = ipywidgets.IntSlider(min=0,max=1000,step=1,value=100,description='ic # ' ,layout=layout,continuous_update=False)
lam = ipywidgets.FloatSlider(min=0,max=0.5,value=0.125,step=0.00001,description='lambda # ' ,layout=layout,continuous_update=False,readout_format='.5f')


t00 = ipywidgets.IntSlider(min=0,max=10_000,step=1,value=0,description='t00 # ' ,layout=layout,continuous_update=False)
t0f = ipywidgets.IntSlider(min=0,max=50_000,step=1,value=t00.value+100,description='t0f # ' ,layout=layout,continuous_update=False)

@ipywidgets.interact(bc1=bc1,bc2=bc2,bc3=bc3,bc4=bc4,ic0=ic0,lam=lam,t00=t00,t0f=t0f,analyze=False,plot=False,save=False)
def compare_solution(bc1,bc2,bc3,bc4,ic0,lam,t00,t0f,analyze=False,plot=False,save=False):
    size = 12  ; mean = 5_00  ;  std  = 2_50 ; step = 10  ; total_step = 1
   
    '''
    generate the adi solution in shape of 
    (frames , rows ,cols)
    '''
    tic = time.time()
    
    grid = generate_grid( size - 2 , bc =(bc1,bc2,bc3,bc4),ic=ic0)
    adi_solution = solve(grid.copy(), iters = t00+step*total_step ,Lambda= lam ,steps=True)
    toc = time.time()
    
    print(f'Numerical solution excuted in {(toc-tic)*1e3}ms')
    
    '''
    model expects scaled input 5d tensor in shape of 
    ( sample size , frames number , rows , cols , channels )
    standard scaling using mean = 500  ,std = 250
    '''
    if analyze :
        # Preprocessing
        model_input = pad_grids(adi_solution,lam)   #pad
        model_input = ( adi_solution[t00:t00+1,:,:] - mean ) / std  #scale 
        model_input = model_input.reshape(1,1,size,size,1)        #reshape to 5d tensor

        
        plot_list =[]
        
        prediction_solutions = {}
        
        tic = time.time() ; 
        
        prediction_solutions[0] = model_input ; 
        
        for i in range(0,total_step+1):
            prediction_solutions[i+1] = loaded_model.predict(prediction_solutions[i])
        
        for i in range(0,total_step+1):
            correct    = (adi_solution[t00+step*(i)],f'$Step={t00+step*(i)}$')
            prediction = ((prediction_solutions[i][0,0,:,:,0]*std )+mean , f'$Step={t00+step*(i)}$')
            plot_list.append(correct)
            plot_list.append(prediction)
            print(i)
        
        
        
        if plot :
            if save :path = f'bc:{bc1},{bc2},{bc3},{bc4},{ic0}-lam:{lam}-t00:{t00}.svg' 
            else: path = None
            print('MAE error:',sklearn.metrics.mean_absolute_error(adi_solution[t00+step*(0+1),1:-1,1:-1],prediction_solutions[0][0,0,1:-1,1:-1,0]))
            
            show_heat_maps(
                *plot_list,
                annotate=True,
                figsize=(50,50),
                rc=(len(plot_list)//2,2),
                cbar_location='bottom'
#             save=path           
            )


interactive(children=(IntSlider(value=600, continuous_update=False, description='bc1 # ', layout=Layout(height…

In [104]:
a =[1,'']
b = [2,'']

tuple(b),tuple(a)

((2, ''), (1, ''))

### Test model

In [4]:
model = tensorflow.keras.models.load_model('12 24 48 96 loss=0.0013.h5')

In [79]:
test_data_batches =generate_data_batches(N=12,
                                          lR=(0,1),
                                          tR=(0,1_000),
                                          max_iters=1000,
                                          seed=192,
                                          batch_size=1,
                                          step_size= 100,
                                          steps=1,
                                          batches=1000,
                                          progress=True)
#                                           save_file = 'N=100x100 batch_size=64 batches=5000 max_iters=1000')

In [47]:
model.evaluate(data_generator(test_data_batches),steps=len(test_data_batches))

1000/1000 [==============================] - 32s 31ms/step - loss: 0.0372 - mse: 0.0108


[0.03718770667910576, 0.010795393027365208]

In [43]:
mean = np.zeros(1000);std= np.zeros(1000)
for bi in test_data_batches:
    mean[int(bi)] = np.mean((test_data_batches_12x12[bi] *250 )+ 500) 
    std[int(bi)]= np.std((test_data_batches_12x12[bi]*250)+500)
    

In [19]:
import matplotlib.pyplot as plt

In [45]:
np.mean(mean)

500.75358987426756